In [94]:
"""This is the data cleaning project"""

'This is the data cleaning project'

In [102]:
import csv
import collections
import re
import pandas_datareader as pdr
from datetime import datetime 

def main():
    print("Initializing...")
    print("Reading Data...")
    filename = input("Please type the name of the file you want to use (without suffix): \n")
    Data = readdata(filename + '.csv')
    print("Formatting Data...")
    Formated_Data = [Data[0]] + cleandata(Data)
    Complete_Data = missing(Formated_Data) #Find and deal with missing data
    Clean_Data = wrong(Complete_Data)
    outfilename = input("Please specify the filename you want to write the cleaned data in: \n")
    with open(outfilename + '.csv', 'w') as fp:
        writer = csv.writer(fp)
        writer.writerows(Clean_Data)  # write data
    print("Output Data Complete!")        
    

In [103]:
#Read Data
def readdata(filename):
    # This function reads data from the original dataset
    with open(filename) as fp:
        reader = csv.reader(fp)
        data = list(reader)
        findata = [i for i in data if i[6] not in ['XL', 'CHK']] #Get rid of stocks not part of S&P 500 index
    return findata

def assetlist():
    # This function returns a set of tickers of S&P500 stocks
    n = readdata()
    k = [ i[6] for i in n]
    return set(k)


In [104]:
# Format Data
def cleanrow(row):
    namepattern = r'[A-Z]*' # To check that all tickers contains only capital letters
    datepattern = r'^(\d{4})-(\d{2})-(\d{2})$' # To check that the dates are in the YYYY-MM-DD format
    matchname = re.search(namepattern, row[6])
    matchdate = re.search(datepattern, row[0])
    
    new_row = [0, 0, 0, 0, 0, 0, 0]
    new_row[0] = row[0] if bool(matchdate) == True else None 
    new_row[1] = round(float(row[1]),2) if row[1] else None
    new_row[2] = round(float(row[2]),2) if row[2] else None
    new_row[3] = round(float(row[3]),2) if row[3] else None
    new_row[4] = round(float(row[4]),2) if row[4] else None
    new_row[5] = int(row[5]) if row[5].isdigit() else None
    new_row[6] = row[6] if bool(matchname) == True else None
    # format the original data, also check for missing data and some cases of wrong data
    # missing and wrong data are replaced by None so we could deal with them more efficiently
    # in the future 
    return new_row

def cleandata(dataset):
    data = dataset[1:]
    for i in range(len(data)):
        data[i] = cleanrow(data[i])
    return data


In [105]:
# Find possible wrong value
# Find missing value

def missing_value(data):
    missing_value = []
    
    for i in range(1, len(data)):
        if None in data[i]:
            missing_value.append(i)
            # find the index of rows with missing value  
        else:
            pass
    return missing_value


In [106]:
# Using webcrawler to fill in the missing data
# and check for the wrong data
import pandas_datareader as pdr
from datetime import datetime

def get_data(ticker, date1, date2, category):
    year1, month1, day1 = int(date1.split('-')[0]), int(date1.split('-')[1]), int(date1.split('-')[2])
    year2, month2, day2 = int(date2.split('-')[0]), int(date2.split('-')[1]), int(date2.split('-')[2])
    data = pdr.get_data_yahoo(symbols = ticker, start = datetime(year1, month1, day1), end = datetime(year2, month2, day2))
    n = data[category]
    # The module does not allow us to query data for a single date, so we query for 2 dates and take the first one
    return round(float(n[0]),2)

def get_missing(row1, row2, data):
    for i in range(len(row1)):
        if row1[i] == None:
            category = data[0][i].capitalize()
            datapoint = get_data(row1[6], row1[0], row2[0], category)
            row1[i] = datapoint
        else:
            pass
    return row1

def missing(data):
    print("Finding Missing Values...")
    missing_rows = missing_value(data)
    print("Fill in Missing Values...")
    for i in missing_rows:
        print(i)
        print(data[i])
        get_missing(data[i], data[i+1], data)
        print(data[i])
    print("Missing Data Resolution Complete!")
    return data


In [107]:
def wrong_value(data):    
    suspicious_high = []
    suspicious_low = []
    suspicious_openclose = []
    
    for i in range(1, len(data)):
        if ((data[i][2] - data[i][3])/data[i][3]) >= 0.4:
            suspicious_high.append(i)
            suspicious_low.append(i)
            # find suspicious high-low gap
        elif abs((data[i][1] - data[i][4])/data[i][4]) >= 0.4:
            suspicious_openclose.append(i)
            # find suspicious open-close gap
        elif data[i][2] < data[i][1] or data[i][2] < data[i][3] or data[i][2] < data[i][4]:
            suspicious_high.append(i)
            # find low, open, close value greater than the high value
        elif data[i][3] > data[i][1] or data[i][3] > data[i][2] or data[i][3] > data[i][4]:
            suspicious_low.append(i)
            # find high, open, close value less than low value
    return suspicious_high, suspicious_low, suspicious_openclose
    
def get_high(data, row1, row2, cat_num = 2):
    datapoint = get_data(row1[6], row1[0], row2[0], data[0][cat_num].capitalize())
    row1[cat_num] = datapoint
    return row1

def get_low(data, row1, row2, cat_num = 3):
    datapoint = get_data(row1[6], row1[0], row2[0], data[0][cat_num].capitalize())
    row1[cat_num] = datapoint
    return row1

def get_open(data, row1, row2, cat_num = 1):
    datapoint = get_data(row1[6], row1[0], row2[0], data[0][cat_num].capitalize())
    row1[cat_num] = datapoint
    return row1

def get_close(data, row1, row2, cat_num = 4):
    datapoint = get_data(row1[6], row1[0], row2[0], data[0][cat_num].capitalize())
    row1[cat_num] = datapoint
    return row1

def wrong(data):
    print("Finding Potential Wrong Values")
    wrong_high_rows, wrong_low_rows, wrong_openclose_rows = wrong_value(data)
    print("Correcting Suspicious 'High' Values")
    for i in wrong_high_rows:
        print(i)
        print(data[i])
        get_high(data, data[i], data[i+1])
        print(data[i])
    print("Suspicious 'High' Values Resolution Complete")
    print("Correcting Suspicious 'Low' Values")
    for i in wrong_low_rows:
        print(i)
        print(data[i])
        get_low(data, data[i], data[i+1])
        print(data[i])
    print("Suspicious 'Low' Values Resolution Complete")
    print("Correcting Suspicious 'Open' and 'Close' Values")
    for i in wrong_openclose_rows:
        print(i)
        print(data[i])
        get_open(data, data[i], data[i+1])
        get_close(data, data[i], data[i+1])
        print(data[i])
    print("Suspicious 'Open' and 'Close' Values Resolution Complete")
    print("Wrong Values Resolution Complete")
    return data


In [108]:
main()

Initializing...
Reading Data...
Please type the name of the file you want to use (without suffix): 
all_stocks_5yr
Formatting Data...
Finding Missing Values...
Fill in Missing Values...
82950
['2017-07-26', None, None, None, 69.08, 3, 'BHF']
['2017-07-26', 69.08, 69.08, 69.08, 69.08, 3, 'BHF']
164476
['2015-07-17', None, 88.76, 88.24, 88.72, 2056819, 'DHR']
['2015-07-17', 67.08, 88.76, 88.24, 88.72, 2056819, 'DHR']
164599
['2016-01-12', None, None, None, 88.55, 0, 'DHR']
['2016-01-12', 67.95, 67.95, 66.26, 88.55, 0, 'DHR']
203818
['2015-07-17', None, 48.49, 47.85, 47.92, 1246786, 'ES']
['2015-07-17', 48.3, 48.49, 47.85, 47.92, 1246786, 'ES']
238574
['2016-07-01', None, None, None, 49.54, 0, 'FTV']
['2016-07-01', 49.4, 54.34, 48.4, 49.54, 0, 'FTV']
433121
['2015-07-17', None, 47.31, 46.83, 46.99, 1229513, 'O']
['2015-07-17', 47.2, 47.31, 46.83, 46.99, 1229513, 'O']
433244
['2016-01-12', None, None, None, 52.43, 0, 'O']
['2016-01-12', 52.52, 52.59, 51.76, 52.43, 0, 'O']
477336
['2015-06-

In [46]:
class Equity:
    def __init__(self, name):
        self.name = name
        self.history = [dataset[0]]+[i for i in cleandata if i[6] == name]
    
    def RoR(self, day1, day2):
        s = [i[4] for i in self.history if i[0] == day1][0]
        St = [i[4] for i in self.history if i[0] == day2][0]
        return (St/s)-1